In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ibge.silver

In [0]:
# Import das funções
from pyspark.sql.functions import col, trim
from pyspark.sql.types import IntegerType, DoubleType

In [0]:
#Lendo DataFrame na Silver
pop_sexo_idade_df = spark.table("ibge.bronze.pop_sexo_idade")

In [0]:
# Remover header residual
pop_sexo_idade_linha_df = (
    pop_sexo_idade_df
    .filter(col("NC") != "Nível Territorial (Código)")
)

# Padronização + Cast
pop_sexo_idade_final_df = (
    pop_sexo_idade_linha_df

    # Renomeando conforme JSON
    .withColumnRenamed("NC", "cod_nivel_territorial")
    .withColumnRenamed("NN", "nivel_territorial")
    .withColumnRenamed("MN", "unid_medida")
    .withColumnRenamed("V", "valor")
    .withColumnRenamed("D1C", "cod_municipio")
    .withColumnRenamed("D1N", "municipio")
    .withColumnRenamed("D2N", "variavel")
    .withColumnRenamed("D3N", "ano")
    .withColumnRenamed("D4N", "sexo")
    .withColumnRenamed("D5N", "idade")
    .withColumnRenamed("D6N", "forma_declaracao_idade")

    # Conversões de tipo
    .withColumn("cod_nivel_territorial", col("cod_nivel_territorial").cast(IntegerType()))
    .withColumn("cod_municipio", col("cod_municipio").cast(IntegerType()))
    .withColumn("ano", col("ano").cast(IntegerType()))
    .withColumn("valor", col("valor").cast(DoubleType()))

    # Trim nas strings
    .withColumn("nivel_territorial", trim(col("nivel_territorial")))
    .withColumn("unid_medida", trim(col("unid_medida")))
    .withColumn("municipio", trim(col("municipio")))
    .withColumn("variavel", trim(col("variavel")))
    .withColumn("sexo", trim(col("sexo")))
    .withColumn("idade", trim(col("idade")))
    .withColumn("forma_declaracao_idade", trim(col("forma_declaracao_idade")))
)

In [0]:
# Caminho gerenciado por tabela
tabela_silver = "ibge.silver.pop_sexo_idade"

(
    pop_sexo_idade_final_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(tabela_silver)
)